#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression as SkLinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [2]:
#!pip install folium

Загрузим данные из csv файла в датафрейм.

In [3]:
# Cell 3: Загрузка данных вместо распаковки 
df = pd.read_csv('input_data.csv', sep=';')
print(f"Данные загружены. Размер: {df.shape}")
print(f"Колонки: {df.columns.tolist()}")


Данные загружены. Размер: (11358150, 15)
Колонки: ['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'geo_lat', 'geo_lon', 'building_type', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id']


In [4]:
# Cell 4: Просмотр данных 
print("Первые 5 строк данных:")
print(df.head())
print("\nИнформация о данных:")
print(df.info())
print(df.shape[0],  df[df["building_type"] < 1].shape[0])

Первые 5 строк данных:
         date     price  level  levels  rooms  area  kitchen_area    geo_lat  \
0  2021-01-01   2451300     15      31      1  30.3           0.0  56.780112   
1  2021-01-01   1450000      5       5      1  33.0           6.0  44.608154   
2  2021-01-01  10700000      4      13      3  85.0          12.0  55.540060   
3  2021-01-01   3100000      3       5      3  82.0           9.0  44.608154   
4  2021-01-01   2500000      2       3      1  30.0           9.0  44.738685   

     geo_lon  building_type  object_type  postal_code  street_id  id_region  \
0  60.699355              0            2     620000.0        NaN         66   
1  40.138381              0            0     385000.0        NaN          1   
2  37.725112              3            0     142701.0   242543.0         50   
3  40.138381              0            0     385000.0        NaN          1   
4  37.713668              3            2     353960.0   439378.0         23   

    house_id  
0  163

Отобразим на карте координаты наших построек.

In [5]:
# Cell 5: Отображение на карте
import folium
from IPython.display import display

map_df = df.head(1000).copy()

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# Добавляем точки на карту
for idx, row in map_df.iterrows():
    folium.Marker(
        location=[row['geo_lat'], row['geo_lon']],
        popup=f"Price: {row['price']} rub"
    ).add_to(m)

#display(m)


# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [6]:
# Cell 6: Функция расчета расстояния Хаверсина и создание признаков
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Расчет расстояния между двумя точками на сфере (Земле) 
    с использованием формулы Хаверсина
    """
    R = 6371  # Радиус Земли в км
    
    # Преобразование градусов в радианы
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Разница координат
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Формула Хаверсина
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    return R * c

# Координаты центров городов
MOSCOW_CENTER = (55.7558, 37.6173)  # Москва
SPB_CENTER = (59.9343, 30.3351)     # Санкт-Петербург

print("Создание географических признаков...")

# Создаем признаки для Москвы
df['distance_to_moscow'] = haversine_distance(
    df['geo_lat'], df['geo_lon'], 
    MOSCOW_CENTER[0], MOSCOW_CENTER[1]
)
df['is_Moscow'] = df['distance_to_moscow'] <= 20

# Создаем признаки для Санкт-Петербурга
df['distance_to_spb'] = haversine_distance(
    df['geo_lat'], df['geo_lon'],
    SPB_CENTER[0], SPB_CENTER[1]
)
df['is_Saint_Peterburg'] = df['distance_to_spb'] <= 20

print(f"Создано признаков: is_Moscow и is_Saint_Peterburg")
print(f"Квартир в Москве: {df['is_Moscow'].sum()}")
print(f"Квартир в СПб: {df['is_Saint_Peterburg'].sum()}")



Создание географических признаков...
Создано признаков: is_Moscow и is_Saint_Peterburg
Квартир в Москве: 1027378
Квартир в СПб: 875985


**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [7]:
# Cell 7: Удаление ненужных признаков
cols_to_drop = ['geo_lat', 'geo_lon', 'object_type', 'postal_code', 
                'street_id', 'id_region', 'house_id', 'distance_to_moscow', 'distance_to_spb']



# Проверяем, какие колонки действительно существуют в данных
existing_cols_to_drop = [col for col in cols_to_drop if col in df.columns]
print(f"Удаляем колонки: {existing_cols_to_drop}")

df = df.drop(columns=existing_cols_to_drop)
print(f"Оставшиеся колонки: {df.columns.tolist()}")

Удаляем колонки: ['geo_lat', 'geo_lon', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id', 'distance_to_moscow', 'distance_to_spb']
Оставшиеся колонки: ['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'building_type', 'is_Moscow', 'is_Saint_Peterburg']


In [8]:
# Фильтрация явно аномальных значений
def clean_data(df):
    # Удаление записей с нулевой или отрицательной площадью
    df = df[df['area'] > 10]  # минимальная разумная площадь
    
    # Удаление записей с нереальными ценами
    df = df[(df['price'] > 100000) & (df['price'] < 500000000)]  # разумный диапазон цен
    
    # Проверка этажности
    df = df[df['level'] <= df['levels']]  # этаж не может быть больше общего числа этажей
    df = df[df['levels'] > 0]  # должен быть хотя бы 1 этаж
    
    return df

df_clean = clean_data(df.copy())
print(f"Размер после очистки: {df_clean.shape}")

Размер после очистки: (11337693, 10)


**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: rooms, building_type, is_Moscow, is_Saint_Petersburg

Числовые: price, level, levels, area, kitchen_area

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [9]:
# Выводим заголовок для наглядности — начинаем анализ типов данных в датафрейме
print("Анализ типов данных по заданию:")
# Выводим типы данных всех столбцов в датафрейме df (например, int64, object, float64 и т.д.)
print(df.dtypes)

# Инициализируем два пустых списка для хранения имён категориальных и числовых признаков
categorical_cols = []   # Список для категориальных признаков
numerical_cols = []     # Список для числовых признаков

# Проходим по всем столбцам датафрейма
for col in df.columns:
    # Пропускаем целевую переменную 'price', так как она не используется как признак
    if col == 'price':
        continue
    # Согласно заданию, 'building_type' — единственный явно указанный категориальный признак
    elif col == 'building_type':
        categorical_cols.append(col)  # Добавляем его в список категориальных
        # Выводим информацию: имя признака и количество уникальных значений (для понимания разнообразия категорий)
        print(f"Категориальный: {col} - {df[col].nunique()} уникальных значений")
    else:
        # Все остальные признаки (кроме 'price' и 'building_type') считаем числовыми
        numerical_cols.append(col)
        print(f"Числовой: {col}")

# Выводим итоговую сводку по типам признаков
print(f"\nИТОГО:")
print(f"Категориальные признаки для кодирования: {categorical_cols}")
print(f"Числовые признаки: {numerical_cols}")

# Проверяем, есть ли категориальные признаки, требующие кодирования
if categorical_cols:
    print(f"\nПрименяем One-Hot Encoding к: {categorical_cols}")
    
    # Создаём экземпляр OneHotEncoder из sklearn
    # drop='first' — удаляем один из фиктивных признаков, чтобы избежать мультиколлинеарности (ловушки фиктивной переменной)
    # sparse_output=False — возвращаем плотный массив (DataFrame), а не разреженную матрицу
    encoder = OneHotEncoder(drop='first', sparse_output=False)
    
    # Применяем обучение и трансформацию только к столбцам из categorical_cols (в данном случае — только 'building_type')
    encoded_cols = encoder.fit_transform(df[categorical_cols])
    
    # Преобразуем результат в pandas DataFrame с осмысленными названиями столбцов
    # encoder.get_feature_names_out(categorical_cols) генерирует имена новых бинарных признаков,
    # например: 'building_type_1', 'building_type_2' и т.д.
    encoded_df = pd.DataFrame(
        encoded_cols, 
        columns=encoder.get_feature_names_out(categorical_cols)
    )
    
    # Удаляем исходные категориальные столбцы из df и добавляем новые закодированные столбцы
    # axis=1 означает объединение по столбцам (горизонтально)
    df = pd.concat([df.drop(columns=categorical_cols), encoded_df], axis=1)
    
    # Выводим информацию о результатах кодирования
    print(f"После кодирования. Размер: {df.shape}")  # Новый размер датафрейма
    # Показываем, какие новые столбцы 
    print(f"Новые колонки от OneHotEncoder: {[col for col in df.columns if 'building_type' in col]}")
else:
    # Если категориальных признаков нет, выводим 
    print("Нет категориальных признаков для кодирования")

# Выводим полный список колонок в итоговом датафрейме
print(f"\nИтоговые колонки: {df.columns.tolist()}")

Анализ типов данных по заданию:
date                   object
price                   int64
level                   int64
levels                  int64
rooms                   int64
area                  float64
kitchen_area          float64
building_type           int64
is_Moscow                bool
is_Saint_Peterburg       bool
dtype: object
Числовой: date
Числовой: level
Числовой: levels
Числовой: rooms
Числовой: area
Числовой: kitchen_area
Категориальный: building_type - 7 уникальных значений
Числовой: is_Moscow
Числовой: is_Saint_Peterburg

ИТОГО:
Категориальные признаки для кодирования: ['building_type']
Числовые признаки: ['date', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'is_Moscow', 'is_Saint_Peterburg']

Применяем One-Hot Encoding к: ['building_type']
После кодирования. Размер: (11358150, 15)
Новые колонки от OneHotEncoder: ['building_type_1', 'building_type_2', 'building_type_3', 'building_type_4', 'building_type_5', 'building_type_6']

Итоговые колонки: ['date', '

**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



In [10]:
# Cell 9: Работа с числовыми признаками (исправленная версия)
print("Работа с числовыми признаками по заданию...")

# 1. Обработка даты согласно заданию (если колонка существует)
if 'date' in df.columns:
    print("1. Обрабатываем колонку 'date'...")
    df['date'] = pd.to_datetime(df['date'])
    first_date = df['date'].min()
    df['days_from_first'] = (df['date'] - first_date).dt.days
    df = df.drop(columns=['date'])
    print(f"Создан признак 'days_from_first'. Удалена колонка 'date'")
else:
    print("1. Колонка 'date' не найдена, пропускаем этот шаг")

print("Данные после обработки даты:")
print(df.head())

# 2. Создаем признак отношения этажа согласно заданию
print("\n2. Создаем признак 'level_ratio'...")
# Проверяем наличие необходимых колонок и обрабатываем возможные ошибки
if 'level' in df.columns and 'levels' in df.columns:
    # Защита от деления на ноль
    df['level_ratio'] = np.where(df['levels'] != 0, df['level'] / df['levels'], 0)
    print(f"Создан признак 'level_ratio'")
    to_drop = ['level', 'levels']
    df = df.drop(columns=to_drop)
    print("Колонки 'level' и 'levels' удалены")
else:
    missing_cols = []
    if 'level' not in df.columns:
        missing_cols.append('level')
    if 'levels' not in df.columns:
        missing_cols.append('levels')
    print(f"   Предупреждение: колонки {missing_cols} не найдены")

print("Данные после создания level_ratio:")
print(df.head())

# 3. Подготовка данных для моделирования
print("\n3. Подготовка данных для моделирования...")

# Определяем числовые признаки для нормализации (все кроме price)
# Исключаем также бинарные признаки is_Moscow и is_Saint_Peterburg
numerical_features_to_scale = [col for col in df.columns 
                              if col != 'price' and 
                              df[col].dtype in ['int64', 'float64', 'int32'] and
                              'building_type' not in col and
                              col not in ['is_Moscow', 'is_Saint_Peterburg']]

print(f"Числовые признаки для нормализации: {numerical_features_to_scale}")

# Разделение на признаки и целевую переменную
X = df.drop(columns=['price'])
y = df['price']

print(f"Размерность признаков: {X.shape}")
print(f"Размерность целевой переменной: {y.shape}")

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=28)

print(f"Обучающая выборка: {X_train.shape}")
print(f"Тестовая выборка: {X_test.shape}")

# 4. Нормализация числовых признаков
print("\n4. Нормализация числовых признаков...")

if numerical_features_to_scale:
    # Применяем StandardScaler отдельно к train и test
    scaler = StandardScaler()
    
    # Обучаем scaler на train данных и преобразуем train
    X_train[numerical_features_to_scale] = scaler.fit_transform(X_train[numerical_features_to_scale])
    
    # Преобразуем test на основе параметров из train
    X_test[numerical_features_to_scale] = scaler.transform(X_test[numerical_features_to_scale])
    
    print("Нормализация завершена!")
    print("Статистика после нормализации:")
    for col in numerical_features_to_scale[:3]:  # Показываем только первые 3 признака
        print(f"  {col}: mean={X_train[col].mean():.2f}, std={X_train[col].std():.2f}")
else:
    print("   Нет числовых признаков для нормализации")

print(f"\nИтоговый размер обучающих данных: {X_train.shape}")
print(f"Итоговый размер тестовых данных: {X_test.shape}")
print(f"Итоговые колонки: {X_train.columns.tolist()}")

# Показываем итоговые данные
print("\nПервые 5 строк обучающих данных после обработки:")
print(X_train.head())

Работа с числовыми признаками по заданию...
1. Обрабатываем колонку 'date'...
Создан признак 'days_from_first'. Удалена колонка 'date'
Данные после обработки даты:
      price  level  levels  rooms  area  kitchen_area  is_Moscow  \
0   2451300     15      31      1  30.3           0.0      False   
1   1450000      5       5      1  33.0           6.0      False   
2  10700000      4      13      3  85.0          12.0      False   
3   3100000      3       5      3  82.0           9.0      False   
4   2500000      2       3      1  30.0           9.0      False   

   is_Saint_Peterburg  building_type_1  building_type_2  building_type_3  \
0               False              0.0              0.0              0.0   
1               False              0.0              0.0              0.0   
2               False              0.0              0.0              1.0   
3               False              0.0              0.0              0.0   
4               False              0.0         

**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

In [11]:
import numpy as np
from numba import njit, prange

@njit(parallel=True, fastmath=True)
def euclidean_distances(X1, X2):
    n1, n2 = X1.shape[0], X2.shape[0]
    D = np.empty((n1, n2), dtype=np.float32)
    for i in prange(n1):
        for j in range(n2):
            s = 0.0
            for k in range(X1.shape[1]):
                diff = X1[i, k] - X2[j, k]
                s += diff * diff
            D[i, j] = np.sqrt(s)
    return D


class KNNRegressor:
    def __init__(self, n_neighbors=5, batch_size=5000):
        self.k = n_neighbors
        self.batch_size = batch_size

    def fit(self, X, y):
        # Преобразуем DataFrame → NumPy float32
        if not isinstance(X, np.ndarray):
            X = X.to_numpy(dtype=np.float32)
        if not isinstance(y, np.ndarray):
            y = y.to_numpy(dtype=np.float32)

        self.X_train = X.astype(np.float32)
        self.y_train = y.astype(np.float32)

    def predict(self, X):
        # Преобразуем DataFrame → NumPy float32
        if not isinstance(X, np.ndarray):
            X = X.to_numpy(dtype=np.float32)
        X = X.astype(np.float32)

        n_test = X.shape[0]
        y_pred = np.zeros(n_test, dtype=np.float32)

        for start in range(0, n_test, self.batch_size):
            end = min(start + self.batch_size, n_test)
            X_batch = X[start:end]

            dists = euclidean_distances(X_batch, self.X_train)

            # Быстрая частичная сортировка по k
            idx = np.argpartition(dists, self.k, axis=1)[:, :self.k]
            y_pred[start:end] = np.mean(self.y_train[idx], axis=1)

        return y_pred



**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [12]:
class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', max_iter=1000, tolerance=1e-4):
        self.learning_rate = learning_rate
        self.optimization = optimization
        self.max_iter = max_iter
        self.tolerance = tolerance
        self.weights = None
        self.bias = None
        self.loss_history = []
    
    def fit(self, X, y):
        #Нормализация входных данных
        X = X.values if isinstance(X, pd.DataFrame) else X
        y = y.values if isinstance(y, pd.Series) else y
        
        # Нормализация признаков
        self.X_mean = np.mean(X, axis=0)
        self.X_std = np.std(X, axis=0) + 1e-8
        X_normalized = (X - self.X_mean) / self.X_std
        
        # Нормализация целевой переменной
        self.y_mean = np.mean(y)
        self.y_std = np.std(y) + 1e-8
        y_normalized = (y - self.y_mean) / self.y_std
        
        n_samples, n_features = X_normalized.shape
        
        # Стабильная инициализация
        self.weights = np.zeros(n_features)
        self.bias = 0.0
        
        # Инициализация для оптимизаторов
        if self.optimization == 'Momentum':
            self.v_w = np.zeros(n_features)
            self.v_b = 0.0
        elif self.optimization == 'AdaGrad':
            self.s_w = np.zeros(n_features) + 1e-8
            self.s_b = 1e-8
        
        previous_loss = float('inf')
        
        for iteration in range(self.max_iter):
            # Прямое распространение
            y_pred = np.dot(X_normalized, self.weights) + self.bias
            
            # Градиенты без регуляризации для стабильности
            error = y_pred - y_normalized
            dw = np.dot(X_normalized.T, error) / n_samples
            db = np.sum(error) / n_samples
            
            # Клиппинг градиентов
            dw = np.clip(dw, -1.0, 1.0)
            db = np.clip(db, -1.0, 1.0)
            
            # Обновление параметров
            current_lr = self.learning_rate / (1 + 0.001 * iteration)  # decay
            
            if self.optimization == 'SGD':
                self.weights -= current_lr * dw
                self.bias -= current_lr * db
            elif self.optimization == 'Momentum':
                self.v_w = 0.9 * self.v_w + current_lr * dw
                self.v_b = 0.9 * self.v_b + current_lr * db
                self.weights -= self.v_w
                self.bias -= self.v_b
            elif self.optimization == 'AdaGrad':
                self.s_w += dw**2
                self.s_b += db**2
                self.weights -= current_lr * dw / np.sqrt(self.s_w)
                self.bias -= current_lr * db / np.sqrt(self.s_b)
            
            # Loss
            loss = np.mean(error**2)
            self.loss_history.append(loss)
            
            if abs(previous_loss - loss) < self.tolerance:
                break
            previous_loss = loss
    
    def predict(self, X):
        X = X.values if isinstance(X, pd.DataFrame) else X
        X_normalized = (X - self.X_mean) / self.X_std
        y_pred_normalized = np.dot(X_normalized, self.weights) + self.bias
        # Денормализация предсказаний
        return y_pred_normalized * self.y_std + self.y_mean

# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.

In [13]:
# 1. Нормализация целевой переменной
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

# 2. Улучшенная обработка выбросов
def improved_clean_data(df):
    df = df.copy()
    
    # Агрессивная фильтрация выбросов
    df = df[df['area'].between(20, 200)]  # разумные пределы площади
    df = df[df['price'].between(500000, 50000000)]  # разумные пределы цены
    df = df[df['kitchen_area'] <= df['area']]  # кухня не больше всей площади
    df = df[df['level'] <= df['levels']]  # этаж не больше общего числа этажей
    df = df[df['levels'] <= 50]  # разумное количество этажей в доме
    
    # Обработка аномальных значений комнат
    df = df[df['rooms'].between(-1, 10)]
    
    return df


In [14]:
def run_final_experiments():
    print("=== ФИНАЛЬНЫЕ ЭКСПЕРИМЕНТЫ С ИСПРАВЛЕННЫМИ МОДЕЛЯМИ ===\n")
    
    # Используем меньшую выборку для стабильности
    sample_size = 100000
    test_size = 20000
    
    # Строгая очистка данных
    def strict_clean_data(df):
        df = df.copy()
        # Убедимся, что все колонки числовые
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        df = df[numeric_cols]
        
        # Фильтрация выбросов
        df = df[df['area'].between(25, 150)]
        df = df[df['price'].between(1000000, 20000000)]
        if 'kitchen_area' in df.columns:
            df = df[df['kitchen_area'].between(5, 50)]
        if 'level_ratio' in df.columns:
            df = df[df['level_ratio'].between(0.05, 0.95)]
        if 'rooms' in df.columns:
            df = df[df['rooms'].between(1, 5)]
        if 'days_from_first' in df.columns:
            df = df[df['days_from_first'].between(0, 1000)]


        df = df[:100000]

        target = "price"
        features = [c for c in df.columns if c != target]

        # Убираем выбросы
        q_low, q_high = df[target].quantile([0.05, 0.95])
        df = df[(df[target] >= q_low) & (df[target] <= q_high)]

        
        return df
    

    # Применяем очистку
    df_strict = strict_clean_data(df)
    print(f"Размер после очистки: {df_strict.shape}")
    
    
    
    # Подготовка данных
    X_final = df_strict.drop(columns=['price'])
    y_final = df_strict['price']
    
    # Убедимся, что все признаки числовые
    X_final = X_final.select_dtypes(include=[np.number])
    
    print(f"Числовые колонки: {X_final.columns.tolist()}")
    
    # Подвыборка
    sample_indices = np.random.choice(len(X_final), size=min(sample_size + test_size, len(X_final)), replace=False)
    X_sample = X_final.iloc[sample_indices]
    y_sample = y_final.iloc[sample_indices]
    
    X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(
        X_sample, y_sample, test_size=test_size, random_state=42
    )
    
    print(f"Размер обучающей выборки: {X_train_final.shape}")
    print(f"Размер тестовой выборки: {X_test_final.shape}")
    print(f"Диапазон цен: {y_train_final.min():,.0f} - {y_train_final.max():,.0f}\n")
    
    def safe_metrics(y_true, y_pred, model_name):
        """Безопасное вычисление метрик"""
        y_true = np.array(y_true, dtype=np.float64)
        y_pred = np.array(y_pred, dtype=np.float64)
        
        # Защита от аномальных значений
        y_pred = np.nan_to_num(y_pred, nan=np.mean(y_true))
        y_pred = np.clip(y_pred, y_true.min() * 0.1, y_true.max() * 10)
        
        mse = mean_squared_error(y_true, y_pred)
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        
        # MAPE
        mask = (y_true != 0) & (np.abs(y_true) > 1000)
        if np.sum(mask) > len(y_true) * 0.5:
            mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
        else:
            mape = np.inf
        
        print(f"{model_name} - MSE: {mse:.2f}, MAE: {mae:.2f}, RMSE: {rmse:.2f}, MAPE: {mape:.1f}%")
        return [mse, mae, rmse, mape]
    
    results = []
    
    # 1. Наша LinearRegression (SGD)
    print("1. Обучение нашей LinearRegression (SGD)...")
    try:
        lr_sgd = LinearRegression(learning_rate=0.1, optimization='SGD', max_iter=300, tolerance=1e-4)
        lr_sgd.fit(X_train_final, y_train_final)
        y_pred_sgd = lr_sgd.predict(X_test_final)
        
        metrics_sgd = safe_metrics(y_test_final, y_pred_sgd, "Our LR (SGD)")
        results.append(["Our LR (SGD)"] + metrics_sgd)
        print(f"   ✅ Успешно. Final loss: {lr_sgd.loss_history[-1]:.4f}\n")
    except Exception as e:
        print(f"   ❌ Ошибка: {e}")
        results.append(["Our LR (SGD)", np.inf, np.inf, np.inf, np.inf])
    
    # 2. Наша LinearRegression (Momentum)
    print("2. Обучение нашей LinearRegression (Momentum)...")
    try:
        lr_momentum = LinearRegression(learning_rate=0.1, optimization='Momentum', max_iter=300, tolerance=1e-4)
        lr_momentum.fit(X_train_final, y_train_final)
        y_pred_momentum = lr_momentum.predict(X_test_final)
        
        metrics_momentum = safe_metrics(y_test_final, y_pred_momentum, "Our LR (Momentum)")
        results.append(["Our LR (Momentum)"] + metrics_momentum)
        print(f"   ✅ Успешно. Final loss: {lr_momentum.loss_history[-1]:.4f}\n")
    except Exception as e:
        print(f"   ❌ Ошибка: {e}")
        results.append(["Our LR (Momentum)", np.inf, np.inf, np.inf, np.inf])
    
    # 3. Наша LinearRegression (AdaGrad)
    print("3. Обучение нашей LinearRegression (AdaGrad)...")
    try:
        lr_adagrad = LinearRegression(learning_rate=0.1, optimization='AdaGrad', max_iter=300, tolerance=1e-4)
        lr_adagrad.fit(X_train_final, y_train_final)
        y_pred_adagrad = lr_adagrad.predict(X_test_final)
        
        metrics_adagrad = safe_metrics(y_test_final, y_pred_adagrad, "Our LR (AdaGrad)")
        results.append(["Our LR (AdaGrad)"] + metrics_adagrad)
        print(f"   ✅ Успешно. Final loss: {lr_adagrad.loss_history[-1]:.4f}\n")
    except Exception as e:
        print(f"   ❌ Ошибка: {e}")
        results.append(["Our LR (AdaGrad)", np.inf, np.inf, np.inf, np.inf])
    
    # 4. Библиотечный LinearRegression
    print("4. Обучение библиотечного LinearRegression...")
    try:
        lr_sklearn = SkLinearRegression()
        lr_sklearn.fit(X_train_final, y_train_final)
        y_pred_sklearn = lr_sklearn.predict(X_test_final)
        
        metrics_sklearn = safe_metrics(y_test_final, y_pred_sklearn, "Sklearn LR")
        results.append(["Sklearn LinearRegression"] + metrics_sklearn)
        print("   ✅ Успешно\n")
    except Exception as e:
        print(f"   ❌ Ошибка: {e}")
        results.append(["Sklearn LinearRegression", np.inf, np.inf, np.inf, np.inf])
    
    # 5. Наш KNNRegressor
    print("5. Обучение нашего KNNRegressor...")
    try:
        knn_our = KNNRegressor(n_neighbors=3)
        knn_our.fit(X_train_final, y_train_final)
        y_pred_knn = knn_our.predict(X_test_final)
        
        metrics_knn = safe_metrics(y_test_final, y_pred_knn, "Our KNN")
        results.append(["Our KNN"] + metrics_knn)
        print("   ✅ Успешно\n")
    except Exception as e:
        print(f"   ❌ Ошибка: {e}")
        results.append(["Our KNN", np.inf, np.inf, np.inf, np.inf])
    
    # 6. Библиотечный KNN
    print("6. Обучение библиотечного KNeighborsRegressor...")
    try:
        knn_sklearn = KNeighborsRegressor(n_neighbors=5)
        knn_sklearn.fit(X_train_final, y_train_final)
        y_pred_sklearn_knn = knn_sklearn.predict(X_test_final)
        
        metrics_sklearn_knn = safe_metrics(y_test_final, y_pred_sklearn_knn, "Sklearn KNN")
        results.append(["Sklearn KNN"] + metrics_sklearn_knn)
        print("   ✅ Успешно\n")
    except Exception as e:
        print(f"   ❌ Ошибка: {e}")
        results.append(["Sklearn KNN", np.inf, np.inf, np.inf, np.inf])
    
    # Вывод результатов
    print("="*80)
    print("ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ:")
    print("="*80)
    
    results_df = pd.DataFrame(results, columns=["Model", "MSE", "MAE", "RMSE", "MAPE"])
    
    def format_metric(x):
        if x == np.inf: 
            return "FAILED"
        elif x > 1e12: 
            return f"{x/1e12:.2f}T"
        elif x > 1e9: 
            return f"{x/1e9:.2f}B"
        elif x > 1e6: 
            return f"{x/1e6:.2f}M"
        elif x > 1e3: 
            return f"{x/1e3:.2f}K"
        else: 
            return f"{x:.2f}"
    
    display_df = results_df.copy()
    for col in ['MSE', 'MAE', 'RMSE']:
        display_df[col] = display_df[col].apply(format_metric)
    display_df['MAPE'] = results_df['MAPE'].apply(lambda x: f"{x:.1f}%" if x != np.inf else "FAILED")
    
    print(display_df.to_string(index=False))
    
    # Анализ
    working_models = results_df[results_df['MSE'] != np.inf]
    
    if len(working_models) > 0:
        best_mse_idx = working_models['MSE'].idxmin()
        best_model = working_models.loc[best_mse_idx, 'Model']
        best_mse = working_models.loc[best_mse_idx, 'MSE']
        
        print(f"\n🏆 Лучшая модель: {best_model} (MSE: {format_metric(best_mse)})")
        
        our_success = [m for m in working_models['Model'] if 'Our' in m]
        sklearn_success = [m for m in working_models['Model'] if 'Sklearn' in m]
        
        print(f"✅ Наши рабочие модели: {', '.join(our_success) if our_success else 'НЕТ'}")
        print(f"✅ Библиотечные модели: {', '.join(sklearn_success)}")
    else:
        print("\n❌ Все модели не сработали")
    
    return results_df

# Запускаем финальные эксперименты
final_results = run_final_experiments()

=== ФИНАЛЬНЫЕ ЭКСПЕРИМЕНТЫ С ИСПРАВЛЕННЫМИ МОДЕЛЯМИ ===

Размер после очистки: (90283, 12)
Числовые колонки: ['rooms', 'area', 'kitchen_area', 'building_type_1', 'building_type_2', 'building_type_3', 'building_type_4', 'building_type_5', 'building_type_6', 'days_from_first', 'level_ratio']
Размер обучающей выборки: (70283, 11)
Размер тестовой выборки: (20000, 11)
Диапазон цен: 1,500,000 - 12,052,490

1. Обучение нашей LinearRegression (SGD)...
Our LR (SGD) - MSE: 4238757739365.64, MAE: 1540112.07, RMSE: 2058824.36, MAPE: 40.5%
   ✅ Успешно. Final loss: 0.7525

2. Обучение нашей LinearRegression (Momentum)...
Our LR (Momentum) - MSE: 4230729459134.44, MAE: 1537523.66, RMSE: 2056873.71, MAPE: 40.5%
   ✅ Успешно. Final loss: 0.7516

3. Обучение нашей LinearRegression (AdaGrad)...
Our LR (AdaGrad) - MSE: 4234560384309.79, MAE: 1539279.06, RMSE: 2057804.75, MAPE: 40.5%
   ✅ Успешно. Final loss: 0.7519

4. Обучение библиотечного LinearRegression...
Sklearn LR - MSE: 4227969096457.33, MAE: 15

In [15]:
# ==============================================
# 🚀 Эксперимент: Быстрый KNN-регрессор без sklearn
# ==============================================
import numpy as np
import pandas as pd
from numba import njit, prange
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# ---------- Быстрая евклидова метрика ----------
@njit(parallel=True, fastmath=True)
def euclidean_distances(X1, X2):
    n1, n2 = X1.shape[0], X2.shape[0]
    D = np.empty((n1, n2), dtype=np.float32)
    for i in prange(n1):
        for j in range(n2):
            s = 0.0
            for k in range(X1.shape[1]):
                diff = X1[i, k] - X2[j, k]
                s += diff * diff
            D[i, j] = np.sqrt(s)
    return D

# ---------- Наш ускоренный KNN-регрессор ----------
class FastKNNRegressor:
    def __init__(self, n_neighbors=3, batch_size=5000):
        self.k = n_neighbors
        self.batch_size = batch_size

    def fit(self, X, y):
        if not isinstance(X, np.ndarray):
            X = X.to_numpy(dtype=np.float32)
        if not isinstance(y, np.ndarray):
            y = y.to_numpy(dtype=np.float32)
        self.X_train = X.astype(np.float32)
        self.y_train = y.astype(np.float32)

    def predict(self, X):
        if not isinstance(X, np.ndarray):
            X = X.to_numpy(dtype=np.float32)
        X = X.astype(np.float32)

        n_test = X.shape[0]
        y_pred = np.zeros(n_test, dtype=np.float32)

        for start in range(0, n_test, self.batch_size):
            end = min(start + self.batch_size, n_test)
            X_batch = X[start:end]

            dists = euclidean_distances(X_batch, self.X_train)
            idx = np.argpartition(dists, self.k, axis=1)[:, :self.k]
            nearest_dists = np.take_along_axis(dists, idx, axis=1)
            nearest_vals = self.y_train[idx]

            weights = 1.0 / (nearest_dists + 1e-6)
            weighted_avg = np.sum(nearest_vals * weights, axis=1) / np.sum(weights, axis=1)
            y_pred[start:end] = weighted_avg

        return y_pred

# ---------- Подготовка данных ----------

data = df.copy()
print("✅ Используется существующий DataFrame 'df'")

data = data[:100000]

target = "price"
features = [c for c in data.columns if c != target]

# Убираем выбросы
q_low, q_high = data[target].quantile([0.05, 0.95])
data = data[(data[target] >= q_low) & (data[target] <= q_high)]

# Логарифмируем целевую переменную
data[target] = np.log1p(data[target])

# Масштабирование признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(data[features])
y = data[target].values

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=28)

# ---------- Обучение и предсказание ----------
print("\n🔹 Обучение FastKNNRegressor...")
knn = FastKNNRegressor(n_neighbors=3, batch_size=200)
knn.fit(X_train, y_train)

print("🔹 Предсказание...")
y_pred_log = knn.predict(X_test)

# Обратное преобразование из логарифма
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

# ---------- Оценка ----------
mape = mean_absolute_percentage_error(y_true, y_pred) * 100
print(f"\n✅ FastKNNRegressor завершён. MAPE = {mape:.2f}%")

# Примеры предсказаний
pd.DataFrame({
    "y_true": y_true[:10],
    "y_pred": y_pred[:10],
    "error_%": np.abs(y_true[:10] - y_pred[:10]) / y_true[:10] * 100
})


✅ Используется существующий DataFrame 'df'

🔹 Обучение FastKNNRegressor...
🔹 Предсказание...

✅ FastKNNRegressor завершён. MAPE = 35.46%


,y_true,y_pred,error_%
0,3600000.0,4364968.00,21.249111
1,1500000.0,4983367.00,232.224467
2,4959503.0,3802997.75,23.318975
3,5356200.0,5094203.00,4.891472
4,4000000.0,5576520.50,39.413012
5,6130000.0,6698663.00,9.276721
6,2750000.0,4864572.00,76.893527
7,3190000.0,2777607.75,12.927657
8,8400517.0,9958958.00,18.551727
9,4590000.0,4824981.00,5.119412
